In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
%matplotlib inline
import matplotlib.pyplot as plt


In [4]:
# Load the data
# file_path = Path('Average Temperature of Cities.csv')
file_path = Path('Annual Average AZ Temps 1970-2022.csv')
file = Path('CO2 Emissions from Fossil Fuels- Fuel Total.csv')
df = pd.read_csv(file_path)
df.head()

,State,Year,Degrees Fahrenheit,Anomaly
0,Arizona,1970,59.0,-0.8
1,Arizona,1971,58.2,-1.6
2,Arizona,1972,59.6,-0.2
3,Arizona,1973,58.4,-1.4
4,Arizona,1974,59.5,-0.3


In [5]:
df_file = pd.read_csv(file)
df_file.head()

,State,Year,Coal,Petroleum Products,Natural Gas
0,Arizona,1970,0.8,13.4,10.7
1,Arizona,1971,0.8,14.3,11.8
2,Arizona,1972,0.7,16.8,12.6
3,Arizona,1973,0.9,21.7,11.8
4,Arizona,1974,4.6,21.4,10.7


In [6]:
df = df.drop(columns = 'State')
df.head()

,Year,Degrees Fahrenheit,Anomaly
0,1970,59.0,-0.8
1,1971,58.2,-1.6
2,1972,59.6,-0.2
3,1973,58.4,-1.4
4,1974,59.5,-0.3


In [7]:
df_file = df_file.drop(columns = 'State')

In [8]:
df_file.head()

,Year,Coal,Petroleum Products,Natural Gas
0,1970,0.8,13.4,10.7
1,1971,0.8,14.3,11.8
2,1972,0.7,16.8,12.6
3,1973,0.9,21.7,11.8
4,1974,4.6,21.4,10.7


In [9]:
df = pd.merge(df, df_file, on = 'Year')

In [10]:
df = df.drop(columns = 'Anomaly')

In [11]:
df.tail()

,Year,Degrees Fahrenheit,Coal,Petroleum Products,Natural Gas
45,2015,61.8,36.9,38.8,19.3
46,2016,62.2,30.9,40.2,19.7
47,2017,63.0,32.0,40.8,17.7
48,2018,62.3,31.7,41.2,21.1
49,2019,60.3,24.6,42.3,25.6


In [12]:
df.columns

Index(['Year', 'Degrees Fahrenheit', 'Coal', 'Petroleum Products',
       'Natural Gas'],
      dtype='object')

In [13]:
X = df[['Year', 'Coal', 'Petroleum Products',
       'Natural Gas']].copy()

X = X.dropna()
y=df['Degrees Fahrenheit']
X.head()

,Year,Coal,Petroleum Products,Natural Gas
0,1970,0.8,13.4,10.7
1,1971,0.8,14.3,11.8
2,1972,0.7,16.8,12.6
3,1973,0.9,21.7,11.8
4,1974,4.6,21.4,10.7


In [14]:
X.dropna()

,Year,Coal,Petroleum Products,Natural Gas
0,1970,0.8,13.4,10.7
1,1971,0.8,14.3,11.8
2,1972,0.7,16.8,12.6
3,1973,0.9,21.7,11.8
4,1974,4.6,21.4,10.7
5,1975,8.8,20.8,8.6
6,1976,13.3,21.1,9.4
7,1977,17.1,24.2,9.2
8,1978,15.2,24.3,9.8
9,1979,23.4,23.3,9.5


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [16]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X_train, y_train)

score = model.score(X, y)
print(f"R2 Score: {score}")

R2 Score: 0.5668356048328688


In [17]:
y_pred = model.predict(X_test)

In [18]:
y_pred

array([59.64967848, 61.05705153, 60.69334968, 61.36810416, 59.86372505,
       61.48760132, 60.39712908, 60.3260572 , 60.77654944, 60.0115347 ,
       59.6201457 , 59.05886093, 61.03928431])

In [19]:
df.columns

Index(['Year', 'Degrees Fahrenheit', 'Coal', 'Petroleum Products',
       'Natural Gas'],
      dtype='object')

In [20]:
test = pd.DataFrame({'Year':[2050],
       'Coal':[30], 'Petroleum Products':[40],
       'Natural Gas':[25]})


In [21]:
model.predict(test)

array([62.90401421])

In [22]:
df.corr()

,Year,Degrees Fahrenheit,Coal,Petroleum Products,Natural Gas
Year,1.000000,0.757579,0.773944,0.952765,0.721341
Degrees Fahrenheit,0.757579,1.000000,0.607625,0.738825,0.503674
Coal,0.773944,0.607625,1.000000,0.703621,0.286516
Petroleum Products,0.952765,0.738825,0.703621,1.000000,0.780433
Natural Gas,0.721341,0.503674,0.286516,0.780433,1.000000


In [23]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [30]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='warn', penalty='12', random_state=42, solver='lbfgs', tol=0.0001, warm_start=False)

LogisticRegression(multi_class='warn', penalty='12', random_state=42)

In [31]:
classifier.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

In [28]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

NameError: name 'predictions' is not defined

In [32]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

ValueError: continuous is not supported